In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
answers = pd.read_csv('./data/Answer.csv')
survey_2020 = pd.read_csv('./data/2020-survey.csv')
correct_columns = [1, 2, 3, 5, 6, 7, 8, 9, 13, 17, 18, 19, 30, 32, 33, 34, 93, 97, 99, 100, 117, 118]

In [35]:
unused_questions_removed = survey_2020.columns[survey_2020.columns.str.startswith('q')]
survey_2020 = survey_2020[unused_questions_removed]


## Filter out unrelevant questions

In [36]:
answers = answers[answers['QuestionID'].isin(correct_columns)]

## Create survey DataFrame

We want a data frame where each row will represent one users answers to one survey. We create a empty DataFrame and fill it out with one row for each unique UserID found in the answers table.

In [37]:
surveys = pd.DataFrame(columns=['q1', 'q2', 'q3', 'q5', 'q6', 'q7', 'q8', 'q9', 'q13', 'q17', 'q18', 'q19', 'q30', 'q32', 'q33', 'q34', 'q93', 'q97', 'q99', 'q100', 'q117', 'q118' 'Year'])
user_ids = answers['UserID'].unique()
surveys['UserID'] = user_ids

In order to fill out the survey DataFrame we have to iterate through each row in the answers table and set the values in the corresponding row in surveys.

In [38]:
for answer in answers.iterrows():
    user_id = answer[1][2]
    question_id = 'q' + str(answer[1][3])
    answer_text = answer[1][0]
    year = answer[1][1]
    surveys.at[user_id - 1, question_id] = answer_text
    surveys.at[user_id - 1, 'Year'] = year
        
    

Some questions are very similar but does only exists in some years, we choose to combine these questions.

In [39]:
surveys['q17'] = surveys['q17'].combine_first(surveys['q97'])
surveys['q18'] = surveys['q18'].combine_first(surveys['q99'])
surveys['q19'] = surveys['q19'].combine_first(surveys['q100'])

surveys = surveys.drop(['q97', 'q99', 'q100'], axis = 1)


## Pre-processing


Making "Male" = "male" and "Female" = "female":

In [40]:
gender_values = surveys['q2'].value_counts()
print(gender_values)

surveys['q2'] = surveys['q2'].str.lower()
gender_values = surveys['q2'].value_counts()
gender_values

Male                           2830
Female                          914
male                            212
female                          110
-1                               24
                               ... 
femmina                           1
transgender                       1
Androgynous                       1
Female-ish                        1
Non-binary and gender fluid       1
Name: q2, Length: 101, dtype: int64


male                3043
female              1024
-1                    24
nonbinary              9
non-binary             9
                    ... 
nah                    1
androgynous            1
ostensibly male        1
trans-female           1
woman-identified       1
Name: q2, Length: 87, dtype: int64

Seting all other values to 'other':

In [41]:
surveys['q2'] = np.where(((surveys.q2 != 'male') & (surveys.q2 != 'female')),'other',surveys.q2)
gender_values = surveys['q2'].value_counts()
gender_values

male      3043
female    1024
other      151
Name: q2, dtype: int64

Make 'United States' = 'United States of America'

In [50]:
country_values = surveys['q3'].value_counts()
print(country_values)
surveys['q3'] = np.where((surveys.q3 == 'United States of America'),'United States',surveys.q3)
country_values = surveys['q3'].value_counts()
country_values

United States             2604
United Kingdom             482
Canada                     199
Germany                    136
Netherlands                 98
Australia                   73
France                      51
Ireland                     51
India                       50
Brazil                      37
Sweden                      29
Switzerland                 28
Spain                       26
New Zealand                 24
Portugal                    23
Poland                      21
Italy                       19
Belgium                     17
Russia                      16
South Africa                16
Finland                     13
Bulgaria                    13
Mexico                      12
Norway                      12
Austria                     10
Japan                        9
Denmark                      9
Israel                       9
Romania                      9
Pakistan                     7
Greece                       7
Czech Republic               6
Colombia

United States             2604
United Kingdom             482
Canada                     199
Germany                    136
Netherlands                 98
Australia                   73
France                      51
Ireland                     51
India                       50
Brazil                      37
Sweden                      29
Switzerland                 28
Spain                       26
New Zealand                 24
Portugal                    23
Poland                      21
Italy                       19
Belgium                     17
Russia                      16
South Africa                16
Finland                     13
Bulgaria                    13
Mexico                      12
Norway                      12
Austria                     10
Japan                        9
Denmark                      9
Israel                       9
Romania                      9
Pakistan                     7
Greece                       7
Czech Republic               6
Colombia

Make 'Maybe' = 'Possibly'


In [78]:
q33_values = surveys['q33'].value_counts()
print(q33_values)
surveys['q33'] = np.where((surveys.q33 == 'Possibly'),'Maybe',surveys.q33)
country_values = surveys['q33'].value_counts()
country_values

Yes           1237
No             969
Maybe          327
Possibly       301
Don't Know     124
Name: q33, dtype: int64


Yes           1237
No             969
Maybe          628
Don't Know     124
Name: q33, dtype: int64

## Pre process 2020 survey
In order to concat the 2020 survey to our surveys DataFrame we have to do some pre processing.

In [8]:
survey_2020 = survey_2020[['q1', 'q2', 'q3', 'q5', 'q6', 'q7', 'q8', 'q9', 'q13', 'q17', 'q18', 'q19', 'q30', 'q32', 'q33', 'q34']]


## Negative age values
Some age values are negative, there fore those entries will be changed to positive.

In [9]:
surveys['q1'] = surveys['q1'].str.replace('-','')

## Plot mental health issues for males and females

Att göra: Lowercase på alla genders, allt som inte är male eller female ska bli other eller tas bort.

In [331]:
surveys['q2'].sample(20)

3587                    Nonbinary
3744                       Female
2400                      Unicorn
3910                         Male
350                          Male
2747                         male
1545                         Male
1713                         Male
1722                         Male
1145                       Female
1031                         Male
3593    Male (or female, or both)
466                          Male
3811                         Male
3463                       female
76                           Male
2264             Male/genderqueer
3092                         Male
3685                         Male
1284                         Male
Name: q2, dtype: object

In [ ]:
data = {'Gender': ['Male','Female', 'Other'],
        'GDP_Per_Capita': [45000,42000,52000,49000,47000]
       }
  
df = pd.DataFrame(data,columns=['Country','GDP_Per_Capita'])
df.plot(x ='Country', y='GDP_Per_Capita', kind = 'bar')
plt.show()